In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -Uqqq pip
!pip install -qqq torch==2.3.0
!pip install -qqq transformers==4.36.2
!pip install -qqq einops==0.7.0
!pip install -qqq accelerate==0.25.0

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import accelerate
torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="auto", 
    torch_dtype="auto",
    trust_remote_code=True,
)


/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-06-29 13:23:45.998342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-29 13:23:45.998460: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-29 13:23:46.126027: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_py

config.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [4]:

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")


tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [6]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}




In [7]:
# Sample chat interaction
messages = [
    {"role": "user", "content": "My weight is 65kgs, my height is 175cm ,bmi is 21, I am a vegetarian. give me suggestions on what diet should I follow ?"},
    {"role": "user", "content": "What are the pros and cons of taking protein powders?"},
    {"role": "user", "content": "What diet changes can help with managing diabetes?"},
    {"role": "user", "content": "What are the best sources of calcium for someone who is lactose intolerant?"},
    {"role": "user", "content": "What is a ketogenic diet, and is it right for me?"},
]

In [8]:
# Loop through each message and generate a response
for message in messages:
    input_text = f"{message['role']}: {message['content']}\n"
    output = pipe(input_text, **generation_args)
    print(output[0]['generated_text'])
    print('-----------------------------------------------------------------------------\n')


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



# ai: Given your weight, height, and BMI, you are within a healthy weight range. As a vegetarian, it's important to ensure you're getting a balanced diet that includes a variety of nutrients. Here are some dietary suggestions tailored to your needs:

1. **Protein Sources**: Since you're a vegetarian, focus on plant-based proteins. Include lentils, chickpeas, black beans, tofu, tempeh, edamame, and seitan in your meals. Quinoa and buckwheat are also excellent sources of complete protein.

2. **Whole Grains**: Opt for whole grains like brown rice, quinoa, oats, and whole wheat bread. These provide essential nutrients and fiber, which are important for digestion and maintaining a healthy weight.

3. **Fruits and Vegetables**: Aim for a colorful variety of fruits and vegetables to ensure you're getting a wide range of vitamins, minerals, and antioxidants. Berries, leafy greens, cruciferous vegetables (like broccoli and Brussels sprouts), and citrus fruits are particularly nutrient-dense.
